In [126]:
import ollama
from utils.duckduckgo import search
import requests
from bs4 import BeautifulSoup

    
# MODEL = "llama3:8b-instruct-q6_K"
MODEL = "qwen2:7b-instruct-q6_K"
# MODEL = "phi:latest"
EMBEDDING_MODEL = "all-minilm:latest"
OLLAMA_HOST = "http://localhost:11434"
RETRY_COUNT = 5
SELECT_TOP_RESULTS = 3
INDEX_NAME = "sfc_code_preprocess"
ollama_client: ollama.Client = ollama.Client(host=OLLAMA_HOST)

In [244]:
from retry import retry
from typing import Union, Generator
from IPython.display import clear_output, Markdown



def get_search_keywords(question: str):
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "You are an SEO expert. A user will give you a question your duty is to figure it out the search keywords for using in a search engine"
                "such as DuckDuckGo to gather the information. "
                "You must return your answer only  and do not include prologue, prefix or suffix.",
            },
            {"role": "user", "content": question},
        ],
        stream=False,
    )
    return response["message"]["content"].split()


def summarize_document(question, document):
    prompt = f"""Question: {question}

Document: {document}

Summarize context that can directly answer the question into bullets point to write a content.
You must give your answer in Thai language and return your answer only  and do not include prologue, prefix or suffix.

"""
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": "You are an research assistance who summarizes a"
                " given a document scraped from a webpage to answer a given question."
                " You must give your answer in Thai language and do not include prologue, prefix or suffix.",
            },
            {"role": "user", "content": prompt},
        ],
        stream=False,
    )
    return response["message"]["content"]


def get_is_useful_prompt(question, context) -> str:
    prompt = f"""From the question:
{question}

And context {context}

Consider if the context is what the question want or not. You must answer with -1 if it's not; 
0 if it's partially related; 1 it it's related. You must give the answer only and do not include prologue, prefix or suffix."""
    return prompt


def get_summary_prompt(question, document):
    prompt = f"""Question: {question}

Document: {document}

Prepare information that can to answer the question into bullets point to write a content.
Also, consider if the docuement is useful to answer the question.
Your answer should be in JSON format with fields:

"is_useful": 0 if not or 1 if it's useful
"summarize": (string) Summarize of the that can response the question or leave blank if you find it entirely not useful.


Return your answer only and do not include prologue, prefix or suffix.
"""
    return prompt


@retry(tries=5, exceptions=json.JSONDecodeError)
def validate_document(question, document):
    summary_prompt = get_summary_prompt(question, document)
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert in lawfirm who are assigned to consider whether a text data source "
                    "is useful to answer a user question or not. If yes, you will summarize the text "
                    "which corespond user's question for another expert to write answer the user , otherwise, do nothing. "
                    """You answer must be in JSON format with field:
"is_useful": boolean determining whether the source is useful,
"summarize: string your summarization refering the part for the text or empty string if not useful
"""
                    " return your answer only  and do not include prologue, prefix or suffix"
                ),
            },
            {"role": "user", "content": summary_prompt},
        ],
        stream=False,
    )
    return json.loads(response["message"]["content"])


def get_context(question: str) -> str:
    contexts = []

    search_results = search(question)
    for search_result in search_results:

        document = search_result["header"]
        context = validate_document(question, document)


        is_useful = context["is_useful"]
        search_result["is_useful"] = is_useful
        search_result["summarize"] = context["summarize"]
        if is_useful == 1:
            contexts.append(search_result)
        if len(contexts) >= 5:
            break

    line_template = """Header:
    {header}

    Summary:
    {summarize}

    Reference: {url}
    """

    writing_prompt = "\n".join([line_template.format(**context) for context in contexts])
    return writing_prompt

def write_the_output(question: str, context_prompts: str):
    response = ollama_client.chat(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful user's assistance, and your secretary already "
                    "prepared gists from the search engine results for "
                    "you to answer user's question. "
                    "Your duty is to answer the question "
                    "with confidence using the prepared "
                    "data source as a reference. "
                    "You must add your idea to support user to understand sources. "
                    "You must add the reference of data source with URL "
                    "and encourage user to find out more information with it. "
                    "You must answer with the same language that the user uses."
                ),
            },
            {"role": "user", "content": f"""Question: {question}

Context:
{context_prompts}
"""},
        ],
        stream=True,
    )
    return response

def display_answer(answer: Union[Generator[str, None, None], str]):
    if isinstance(answer, Generator):
        cumulative_response = ""
        for c in answer:
            if isinstance(c, dict):
                c = c["message"]["content"]
            print(c, end="", flush=True)
            cumulative_response += c
        clear_output(wait=True)
        display(Markdown(cumulative_response))
    else:
        display(Markdown(answer))

def suplexity(question):
    context_prompts = get_context(question)
    stream = write_the_output(question, context_prompts)
    return stream

display_answer(suplexity("ดูดวงราศีเมถุน"))

ท่านสามารถดูดวงชะตาปี 2567 สำหรับราศีเมถุนผ่านแหล่งข้อมูลต่างๆ ที่มีการอภิปรายและทำนายความรัก การงาน การเงิน และสุขภาพของผู้ที่เป็นราศีเมถุน

1. **สำหรับดวงรายปี 2567:** อาจารย์แทมมี่ เมจิก ทาโร่ต์, ผู้เชี่ยวชาญด้านโหราศาสตร์, ได้สรุปและทำนายความสัมพันธ์ของราศีเมถุนในปี 2567. ท่านสามารถเข้าไปอ่านข้อมูลเพิ่มเติมที่ [นี่](https://www.sanook.com/horoscope/264603/) 

2. **สำหรับการเปลี่ยนแปลงในปี 2024:** ชาวราศีเมถุนมีความคาดหวังและโอกาสในการทำงานที่มากมายอย่างมาก ในช่วงปี 2024 เขามีโอกาสที่จะประสบความสำเร็จ. เพิ่มเติมอ่านได้ที่ [นี่](https://horo2024.com/th/horoscope-gemini-2024/)

3. **การมีจังหวะทองในช่วงต้นปี:** ราศีเมถุนมีโอกาสเจริญก้าวหน้าและดวงดีในช่วงแรกของปี ท่านสามารถอ่านเพิ่มเติมได้ที่ [นี่](https://today.line.me/th/v2/article/1DOrD5B)

4. **ราศีเมถุนมีสัญลักษณ์เป็นคนคู่หรือฝาแฝด:** ข้อมูลเกี่ยวกับราศีเมถุนที่ครอบคลุมผู้ที่เกิดระหว่างวันที่ 15 มิถุนายน - 15 กรกฎาคม. ชาวเมถุนมีสัญลักษณ์เป็นคนคู่หรือฝาแฝด, ซึ่งหมายถึง การตีสองหน้า. เพิ่มเติมอ่านได้ที่ [นี่](https://horoscope.thaiorc.com/zodiac/gemini.php)

5. **การดูดวงปี 2567 สำหรับราศีเมถุน:** โดยหมอลักษณ์ ราชสีห์, มีการทำนายในเรื่องความรัก การงาน การเงิน และสุขภาพของผู้ที่เป็นราศีเมถุน. เพิ่มเติมอ่านได้ที่ [นี่](https://www.sanook.com/horoscope/yenyenxmorluck/gemini/)

ขอแนะนำให้ท่านเข้าไปอ่านเพื่อดูรายละเอียดของข้อมูลและทำนายในแต่ละแหล่ง ซึ่งจะช่วยให้ท่านได้รับความรู้และข้อมูลเกี่ยวกับดวงชะตาของราศีเมถุนมากขึ้น

In [245]:
display_answer(suplexity("สรุปข่าวหุ้นไทย วันนี้"))

สรุปข่าวหุ้นไทยวันนี้ได้ว่า ตลาดหุ้นไทยปิดการซื้อขายที่ระดับ 1,311.78 จุด ลดลงจากวันก่อนประมาณ 4.91 จุดหรือ -0.37% มูลค่าของทั้งหมดที่ซื้อขายในวันนี้อยู่ที่ 38,267.35 ล้านบาท โดยดัชนีหุ้นปรับตัวขึ้นและลงอย่างไม่แน่นอนในแดนลบ แต่มีการเปลี่ยนแปลงของตลาดหุ้นไทยให้ดูข้อมูลเพิ่มเติมได้ที่ [link](https://www.infoquest.co.th/2024/406098)

อย่างไรก็ตาม หุ้นไทยปิดบวก 0.59 จุดในวันนี้ โดยดีดขึ้นระยะสั้นแต่ความผันผวนยังคงมีอยู่ การvaluationของตลาดถูกแล้วแต่ limit ทาง Upside ไม่มากนัก ให้อ่านเพิ่มเติมได้ที่ [link](https://www.infoquest.co.th/stock)

นอกจากนี้ ข้อมูลรวมทั้งตลาด ราคาหลักทรัพย์ สิบอันดับหลักทรัพย์ และมูลค่าการซื้อขายในแต่ละวันสำหรับหุ้นไทยให้อ่านได้ที่ [link](https://www.set.or.th/th/market/news-and-alert/news)

และหากต้องการข้อมูลเพิ่มเติม สามารถเข้าไปดูข่าวและบทความเกี่ยวกับภาวะตลาดหุ้นไทยทั้งหมดได้ที่ [link](https://www.efinancethai.com/) และอีกหนึ่งแหล่งข้อมูลจากธนาคารแห่งประเทศไทยที่รวบรวมภาพรวมของตลาด ราคาหลักทรัพย์ สิบอันดับหลักทรัพย์ และมูลค่าการซื้อขายในแต่ละวันสำหรับหุ้นไทยได้ที่ [link](https://www.set.or.th/th/market/news-and-alert/news)

ขอให้มีความสุขกับการศึกษาและสำรวจข้อมูลเกี่ยวกับตลาดหุ้นไทยของคุณ!